#### Feature Extraction

In [ ]:

import os, json, gc
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint


os.environ.pop("XLA_FLAGS", None)
tf.config.set_soft_device_placement(True)
tf.config.threading.set_intra_op_parallelism_threads(2)
tf.config.threading.set_inter_op_parallelism_threads(2)

gpus = tf.config.list_physical_devices("GPU")
for g in gpus:
    try:
        tf.config.experimental.set_memory_growth(g, True)
    except Exception:
        pass

if gpus:
    try:
        tf.keras.mixed_precision.set_global_policy("mixed_float16")
        print("Mixed precision ON")
    except Exception:
        print("Mixed precision not enabled; continuing in float32.")


OUT_DIR   = r'D:\Research\Custom CNN\Without Augmented'
os.makedirs(OUT_DIR, exist_ok=True)

data_root = r'D:\Research\Custom CNN\Without Augmented\Original Image'  # class subfolders
IMG_SIZE  = (224, 224)
BATCH     = 64
VAL_SPLIT = 0.2
SEED      = 42

FULL_MODEL_H5      = os.path.join(OUT_DIR, "custom_cnn_full.h5")
WEIGHTS_H5         = os.path.join(OUT_DIR, "custom_cnn.weights.h5")
FEAT_EXTRACTOR_H5  = os.path.join(OUT_DIR, "custom_cnn_feature_extractor.h5")
META_JSON          = os.path.join(OUT_DIR, "custom_cnn_meta.json")
CKPT_WEIGHTS       = os.path.join(OUT_DIR, "custom_cnn_best.weights.h5")

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_root, labels="inferred", label_mode="int",
    image_size=IMG_SIZE, batch_size=BATCH,
    validation_split=VAL_SPLIT, subset="training", seed=SEED, shuffle=True
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_root, labels="inferred", label_mode="int",
    image_size=IMG_SIZE, batch_size=BATCH,
    validation_split=VAL_SPLIT, subset="validation", seed=SEED, shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)

def norm(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    return x, y

train_ds = train_ds.map(norm, num_parallel_calls=2).prefetch(2)
val_ds   = val_ds.map(norm,   num_parallel_calls=2).prefetch(2)


aug = tf.keras.Sequential([
    L.RandomFlip("horizontal"),
    L.RandomRotation(0.05),
    L.RandomZoom(0.1),
], name="aug")


def conv_block(x, filters, k=3, s=1, p="same"):
    x = L.Conv2D(filters, k, strides=s, padding=p, use_bias=False)(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    return x

def build_custom_cnn(input_shape=(224,224,3), n_classes=3, feature_dim=256, dropout=0.5):
    inputs = L.Input(shape=input_shape)
    x = aug(inputs)

    x = conv_block(x, 32); x = conv_block(x, 32); x = L.MaxPooling2D(2)(x)
    x = conv_block(x, 64); x = conv_block(x, 64); x = L.MaxPooling2D(2)(x)
    x = conv_block(x, 128); x = conv_block(x, 128); x = L.MaxPooling2D(2)(x)

    gap = L.GlobalAveragePooling2D()(x)
    se  = L.Dense(128//4, activation="relu", dtype="float32")(gap)
    se  = L.Dense(128, activation="sigmoid", dtype="float32")(se)
    x   = L.Multiply()([x, L.Reshape((1,1,128))(se)])

    x = L.GlobalAveragePooling2D(name="gap")(x)
    feat = L.Dense(feature_dim, activation="relu", name="feature_dense", dtype="float32")(x)
    x = L.Dropout(dropout)(feat)
    outputs = L.Dense(n_classes, activation="softmax", name="logits", dtype="float32")(x)

    model = Model(inputs, outputs, name="custom_cnn")
    feat_model = Model(inputs, feat, name="custom_cnn_feature_extractor")
    return model, feat_model

model, feat_model = build_custom_cnn(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    n_classes=num_classes,
    feature_dim=256,
    dropout=0.5
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)


callbacks = [
    ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=3, verbose=1, min_lr=1e-6),
    ModelCheckpoint(
        CKPT_WEIGHTS, monitor="val_accuracy",
        save_best_only=True, save_weights_only=True, verbose=1
    ),
]

EPOCHS = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

val_acc = history.history.get("val_accuracy", [])
if len(val_acc) > 0 and max(val_acc) > val_acc[-1] and os.path.exists(CKPT_WEIGHTS):
    print(f"Loading best weights (val_acc {max(val_acc):.4f} > last {val_acc[-1]:.4f})")
    model.load_weights(CKPT_WEIGHTS)
else:
    print("Keeping final epoch weights (no improvement over last OR no checkpoint).")

model.save(FULL_MODEL_H5)
model.save_weights(WEIGHTS_H5)
feat_model.save(FEAT_EXTRACTOR_H5)

with open(META_JSON, "w") as f:
    json.dump({
        "img_size": IMG_SIZE,
        "num_classes": num_classes,
        "class_names": class_names,
        "feature_dim": 256,
        "epochs_trained": int(len(history.history.get("loss", []))),
        "best_val_accuracy": float(max(val_acc)) if len(val_acc) else None,
        "last_val_accuracy": float(val_acc[-1]) if len(val_acc) else None,
        "out_dir": OUT_DIR
    }, f, indent=2)

print(f"Saved full model (.h5): {FULL_MODEL_H5}")
print(f"Saved weights (.weights.h5): {WEIGHTS_H5}")
print(f"Saved feature extractor (.h5): {FEAT_EXTRACTOR_H5}")
print(f"Saved meta: {META_JSON}")

gc.collect()
tf.keras.backend.clear_session()

c:\Users\Rittik\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Rittik\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Rittik\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/fram

Found 2195 files belonging to 6 classes.
Using 1756 files for training.
Found 2195 files belonging to 6 classes.
Using 439 files for validation.
Epoch 1/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 30s/step - accuracy: 0.4000 - loss: 1.5029 
Epoch 1: val_accuracy improved from None to 0.00000, saving model to D:\Research\Custom CNN\Without Augmented\custom_cnn_best.weights.h5
28/28 ━━━━━━━━━━━━━━━━━━━━ 896s 32s/step - accuracy: 0.4710 - loss: 1.3375 - val_accuracy: 0.0000e+00 - val_loss: 1.8443 - learning_rate: 0.0010
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 38s/step - accuracy: 0.5564 - loss: 1.0974 
Epoch 2: val_accuracy did not improve from 0.00000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1101s 39s/step - accuracy: 0.5598 - loss: 1.0944 - val_accuracy: 0.0000e+00 - val_loss: 2.2533 - learning_rate: 0.0010
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - accuracy: 0.5485 - loss: 1.0704 
Epoch 3: val_accuracy did not improve from 0.00000
28/28 ━━━━━━━━━━━━━━━━━━━━ 820s 29s/step - accuracy: 0.5598 - loss: 1.

Keeping final epoch weights (no improvement over last OR no checkpoint).


Saved full model (.h5): D:\Research\Custom CNN\Without Augmented\custom_cnn_full.h5
Saved weights (.weights.h5): D:\Research\Custom CNN\Without Augmented\custom_cnn.weights.h5
Saved feature extractor (.h5): D:\Research\Custom CNN\Without Augmented\custom_cnn_feature_extractor.h5
Saved meta: D:\Research\Custom CNN\Without Augmented\custom_cnn_meta.json



#### Feature extraction pipeline 

In [ ]:
import os, json, glob, math, random
import numpy as np
import pandas as pd
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, BatchNormalization, Dropout,
    GlobalAveragePooling2D, Dense
)
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_preprocess


SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)
try:
    tf.config.experimental.enable_op_determinism()
except Exception:
    pass  


USE_MIXED_PRECISION = True and bool(tf.config.list_physical_devices('GPU'))
if USE_MIXED_PRECISION:
    try:
        tf.keras.mixed_precision.set_global_policy('mixed_float16')
        print("Mixed precision enabled (global_policy = 'mixed_float16').")
    except Exception:
        print("Could not enable mixed precision; proceeding in float32.")



IMG_SIZE = (224, 224)
BATCH_SIZE = 64
NUM_WORKERS = tf.data.AUTOTUNE


data_root = r'D:\Research\Custom CNN\Without Augmented\Original Image'

OUT_DIR = r'D:\Research\Custom CNN\Without Augmented'


class_dirs = sorted([d for d in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, d))])
if not class_dirs:
    raise RuntimeError(f"No class folders found under {data_root}")

class_indices = {c: i for i, c in enumerate(class_dirs)}
index_to_class = {i: c for c, i in class_indices.items()}

BACKBONE = 'custom'

CUSTOM_WEIGHTS_PATH = 'custom_cnn.weights.h5'
EFF_WEIGHTS = 'imagenet'

OUT_BASENAME = 'features_256d_' + BACKBONE
CSV_PATH = os.path.join(OUT_DIR, f'{OUT_BASENAME}.csv')
PARQUET_PATH = os.path.join(OUT_DIR, f'{OUT_BASENAME}.parquet')
META_JSON = os.path.join(OUT_DIR, f'{OUT_BASENAME}_meta.json')


def build_custom_cnn(num_classes: int, input_shape=(224, 224, 3)):

    inputs = Input(shape=input_shape)
    # Block 1
    x = Conv2D(32, 3, padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2)(x)
    # Block 2
    x = Conv2D(64, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2)(x)
    # Block 3
    x = Conv2D(128, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2)(x)
    
    x = GlobalAveragePooling2D(name='gap', dtype='float32')(x)
    penultimate = Dense(256, activation='relu', dtype='float32', name='feature_dense')(x)  
    x = Dropout(0.5)(penultimate)
    outputs = Dense(num_classes, activation='softmax', name='logits')(x)

    cls_model = Model(inputs, outputs, name='custom_cnn')
    feat_model = Model(inputs, penultimate, name='custom_cnn_feature_extractor')
    return cls_model, feat_model


def build_efficientnet_feature_model(output_dim=256, input_shape=(224, 224, 3), weights='imagenet'):
    
    base = EfficientNetB0(include_top=False, weights=weights, input_shape=input_shape)
    inputs = base.input
    x = base.output
    
    x = GlobalAveragePooling2D(name='gap', dtype='float32')(x)
    penultimate = Dense(output_dim, activation='relu', dtype='float32', name='feature_dense')(x) 
    feat_model = Model(inputs, penultimate, name='efficientnet_feature_extractor')
    return feat_model



if BACKBONE == 'custom':
    cls_model, feature_model = build_custom_cnn(num_classes=len(class_dirs), input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    if CUSTOM_WEIGHTS_PATH and os.path.exists(CUSTOM_WEIGHTS_PATH):
        try:
            
            cls_model.load_weights(CUSTOM_WEIGHTS_PATH, by_name=True, skip_mismatch=True)
            print(f"Loaded custom CNN weights from: {CUSTOM_WEIGHTS_PATH}")
        except Exception as e:
            print(f"Could not load custom weights: {e}\nProceeding with random init (features will be weak).")
    preprocess_fn = lambda x: tf.cast(x, tf.float32) / 255.0
else:
    feature_model = build_efficientnet_feature_model(
        output_dim=256, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), weights=EFF_WEIGHTS
    )
    preprocess_fn = eff_preprocess  

records = []
for cls in class_dirs:
    folder = os.path.join(data_root, cls)
    files = sorted(
        glob.glob(os.path.join(folder, '*.jpg')) +
        glob.glob(os.path.join(folder, '*.jpeg')) +
        glob.glob(os.path.join(folder, '*.png'))
    )
    for fp in files:
        records.append((fp, cls, class_indices[cls]))

if not records:
    raise RuntimeError(f"No images found under {data_root} (searched *.jpg, *.jpeg, *.png).")

paths = [r[0] for r in records]
labels = [r[2] for r in records]
fnames = [os.path.basename(r[0]) for r in records]
N = len(paths)
print(f"Found {N} images across {len(class_dirs)} classes.")


def load_and_preprocess(path, label, fname):
    img_bytes = tf.io.read_file(path)
   
    img = tf.image.decode_image(img_bytes, channels=3, expand_animations=False)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, IMG_SIZE, method='bilinear', antialias=True)
    img = preprocess_fn(img)
    return img, label, fname

ds = tf.data.Dataset.from_tensor_slices((paths, labels, fnames))
ds = ds.map(load_and_preprocess, num_parallel_calls=NUM_WORKERS)
ds = ds.batch(BATCH_SIZE).prefetch(NUM_WORKERS)

all_feats = []
all_labels = []
all_fnames = []

num_batches = math.ceil(N / BATCH_SIZE)
for batch_imgs, batch_labels, batch_names in tqdm(ds, total=num_batches, desc="Extracting"):
    
    feats = feature_model(batch_imgs, training=False)  
    
    feats = tf.cast(feats, tf.float32)
    all_feats.append(feats.numpy())
    all_labels.extend(batch_labels.numpy().tolist())
    all_fnames.extend(batch_names.numpy().astype(str).tolist())

features = np.vstack(all_feats)  # [N, 256]
assert features.shape[0] == N, "Feature count mismatch"

df = pd.DataFrame(features, columns=[f'f{i:03d}' for i in range(features.shape[1])])
df['class_idx'] = all_labels
df['label'] = [index_to_class[i] for i in all_labels]
df['filename'] = all_fnames

df.to_csv(CSV_PATH, index=False)
try:
    df.to_parquet(PARQUET_PATH, index=False)
except Exception as e:
    print(f"Parquet save failed ({e}); CSV still saved.")

with open(META_JSON, 'w') as f:
    json.dump({
        'img_size': IMG_SIZE,
        'backbone': BACKBONE,
        'feature_dim': int(features.shape[1]),
        'class_indices': class_indices,
        'num_images': int(features.shape[0]),
        'data_root': data_root,
        'mixed_precision': bool(USE_MIXED_PRECISION),
    }, f, indent=2)

print(f"Saved CSV: {CSV_PATH}")
if os.path.exists(PARQUET_PATH):
    print(f"Saved Parquet: {PARQUET_PATH}")
print(f"Saved meta: {META_JSON}")


Could not load custom weights: `by_name` only supports loading legacy '.h5' or '.hdf5' files. Received: custom_cnn.weights.h5
Proceeding with random init (features will be weak).
Found 2195 images across 6 classes.


Extracting: 100%|██████████| 35/35 [01:07<00:00,  1.91s/it]


Parquet save failed (Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.); CSV still saved.
Saved CSV: D:\Research\Custom CNN\Without Augmented\features_256d_custom.csv
Saved meta: D:\Research\Custom CNN\Without Augmented\features_256d_custom_meta.json


### ML on custom feature

#### XGboost

In [ ]:
import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
import xgboost as xgb
import joblib


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):  return int(obj)
        if isinstance(obj, np.floating): return float(obj)
        if isinstance(obj, np.ndarray):  return obj.tolist()
        return super().default(obj)


BASE_DIR   = r'D:\Research\Custom CNN\Without Augmented'
CSV_PATH   = r'D:\Research\Custom CNN\Without Augmented\features_256d_custom.csv'
OUT_DIR    = os.path.join(BASE_DIR, "XGB_Custom")
os.makedirs(OUT_DIR, exist_ok=True)

REPORT_OUT     = os.path.join(OUT_DIR, "xgb_custom_report.json")
MODEL_VAL_BEST = os.path.join(OUT_DIR, "xgb_custom_valbest.json")       
MODEL_FINAL    = os.path.join(OUT_DIR, "xgb_custom_final_trainval.json")
SCALER_PATH    = os.path.join(OUT_DIR, "xgb_custom_scaler.joblib")
SEARCH_CSV     = os.path.join(OUT_DIR, "xgb_custom_val_search_results.csv")


df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
if not feat_cols:
    raise ValueError("No feature columns found with names like f0..f255")

X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(np.int64)

if "label" in df.columns:
    class_map = df.sort_values("class_idx")[["class_idx","label"]].drop_duplicates()
    class_names = class_map.set_index("class_idx")["label"].reindex(sorted(class_map["class_idx"])).tolist()
else:
    class_names = [str(i) for i in sorted(np.unique(y))]

n_classes = int(len(np.unique(y)))
rng = 42


X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=rng
)


scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)
joblib.dump(scaler, SCALER_PATH)


dtrain = xgb.DMatrix(X_train_s, label=y_train)
dval   = xgb.DMatrix(X_val_s,   label=y_val)
dtrainval = xgb.DMatrix(np.vstack([X_train_s, X_val_s]), label=np.concatenate([y_train, y_val]))
dtest  = xgb.DMatrix(X_test_s,  label=y_test)


def make_params(**p):
    return {
        "objective": "multi:softprob",
        "num_class": n_classes,
        "tree_method": "hist",
        "eval_metric": ["mlogloss","merror"],
        "seed": rng,
        "eta": p.get("learning_rate", 0.06),
        "max_depth": p.get("max_depth", 4),
        "min_child_weight": p.get("min_child_weight", 1),
        "subsample": p.get("subsample", 1.0),
        "colsample_bytree": p.get("colsample_bytree", 1.0),
        "alpha": p.get("reg_alpha", 0.0),
        "lambda": p.get("reg_lambda", 1.0),
        "gamma": p.get("gamma", 0.0),
    }

def predict_proba_booster(booster, dm):
    
    ntree_limit = getattr(booster, "best_ntree_limit", None)
    if ntree_limit is not None and ntree_limit > 0:
        proba = booster.predict(dm, ntree_limit=ntree_limit)
    else:
        best_it = getattr(booster, "best_iteration", None)
        if best_it is not None:
            proba = booster.predict(dm, iteration_range=(0, best_it + 1))
        else:
            proba = booster.predict(dm)
    return proba

def evaluate_with_reports(dm, y_true, proba, prefix):
    y_pred = np.argmax(proba, axis=1)

    acc   = accuracy_score(y_true, y_pred)
    f1m   = f1_score(y_true, y_pred, average="macro")
    f1w   = f1_score(y_true, y_pred, average="weighted")
    precm = precision_score(y_true, y_pred, average="macro")
    precw = precision_score(y_true, y_pred, average="weighted")
    recm  = recall_score(y_true, y_pred, average="macro")
    recw  = recall_score(y_true, y_pred, average="weighted")

    
    rep = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    pd.DataFrame(rep).transpose().to_csv(os.path.join(OUT_DIR, f"{prefix}_classification_report.csv"))

    
    cm = confusion_matrix(y_true, y_pred)
    pd.DataFrame(cm, index=class_names, columns=class_names).to_csv(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.csv"))
    plt.figure(figsize=(6,5))
    plt.imshow(cm, interpolation='nearest')
    plt.title(f"Confusion Matrix — {prefix}")
    plt.colorbar()
    ticks = np.arange(len(class_names))
    plt.xticks(ticks, class_names, rotation=45, ha="right")
    plt.yticks(ticks, class_names)
    th = cm.max() / 2 if cm.max() > 0 else 0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, str(cm[i, j]), ha="center", va="center",
                     color="white" if cm[i, j] > th else "black")
    plt.tight_layout(); plt.ylabel("True"); plt.xlabel("Pred")
    plt.savefig(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.png"), dpi=160, bbox_inches="tight")
    plt.close()

    metrics_extra = {}
    if n_classes > 1:
        y_bin = label_binarize(y_true, classes=np.arange(n_classes))
        fpr, tpr, roc_auc, prec, rec, ap = {}, {}, {}, {}, {}, {}

        for c in range(n_classes):
            fpr[c], tpr[c], _ = roc_curve(y_bin[:, c], proba[:, c])
            roc_auc[c] = auc(fpr[c], tpr[c])
            prec[c], rec[c], _ = precision_recall_curve(y_bin[:, c], proba[:, c])
            ap[c] = average_precision_score(y_bin[:, c], proba[:, c])

        fpr["micro"], tpr["micro"], _ = roc_curve(y_bin.ravel(), proba.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        prec["micro"], rec["micro"], _ = precision_recall_curve(y_bin.ravel(), proba.ravel())
        ap["micro"] = average_precision_score(y_bin, proba, average="micro")

        all_fpr = np.unique(np.concatenate([fpr[c] for c in range(n_classes)]))
        mean_tpr = np.zeros_like(all_fpr)
        for c in range(n_classes):
            mean_tpr += np.interp(all_fpr, fpr[c], tpr[c])
        mean_tpr /= n_classes
        roc_auc["macro"] = auc(all_fpr, mean_tpr)
        ap["macro"] = float(np.mean([ap[c] for c in range(n_classes)]))

        
        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for xi, yi in zip(fpr[key], tpr[key]):
                rows.append({"curve": f"ROC_{key}", "fpr": float(xi), "tpr": float(yi)})
        for xi, yi in zip(all_fpr, mean_tpr):
            rows.append({"curve": "ROC_macro", "fpr": float(xi), "tpr": float(yi)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_roc_points.csv"), index=False)

        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for pi, ri in zip(prec[key], rec[key]):
                rows.append({"curve": f"PR_{key}", "precision": float(pi), "recall": float(ri)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_pr_points.csv"), index=False)

        
        plt.figure(figsize=(7,6))
        for c in range(n_classes):
            plt.plot(fpr[c], tpr[c], lw=1.2, label=f"{class_names[c]} (AUC={roc_auc[c]:.3f})")
        plt.plot(fpr["micro"], tpr["micro"], lw=2, linestyle="--", label=f"micro (AUC={roc_auc['micro']:.3f})")
        plt.plot([0,1],[0,1],"k--", lw=1)
        plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {prefix}")
        plt.legend(loc="lower right", fontsize=8)
        plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.png"), dpi=160); plt.close()

        plt.figure(figsize=(7,6))
        for c in range(n_classes):
            plt.plot(rec[c], prec[c], lw=1.2, label=f"{class_names[c]} (AP={ap[c]:.3f})")
        plt.plot(rec["micro"], prec["micro"], lw=2, linestyle="--", label=f"micro (AP={ap['micro']:.3f})")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {prefix}")
        plt.legend(loc="lower left", fontsize=8)
        plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.png"), dpi=160); plt.close()

        metrics_extra = {
            "roc_auc_per_class": {class_names[c]: float(roc_auc[c]) for c in range(n_classes)},
            "roc_auc_micro": float(roc_auc["micro"]),
            "roc_auc_macro": float(roc_auc["macro"]),
            "ap_per_class": {class_names[c]: float(ap[c]) for c in range(n_classes)},
            "ap_micro": float(ap["micro"]),
            "ap_macro": float(ap["macro"]),
        }

    return {
        "accuracy": float(acc),
        "f1_macro": float(f1m),
        "f1_weighted": float(f1w),
        "precision_macro": float(precm),
        "precision_weighted": float(precw),
        "recall_macro": float(recm),
        "recall_weighted": float(recw),
        **metrics_extra
    }

PARAMS = {
    "learning_rate":    [0.03, 0.06],
    "max_depth":        [3, 4, 5],
    "min_child_weight": [1, 3],
    "subsample":        [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "reg_alpha":        [0.0, 1e-3],
    "reg_lambda":       [1.0],
    "gamma":            [0.0],
}
BASE_N_ESTIMATORS = 2000
EARLY_STOP_ROUNDS = 50

keys = list(PARAMS.keys())
grid = [dict(zip(keys, vals)) for vals in product(*[PARAMS[k] for k in keys])]

search_rows = []
best = {"acc": -1.0, "params": None, "booster": None, "best_round": None}

for p in grid:
    params = make_params(**p)
    booster = xgb.train(
        params, dtrain, num_boost_round=BASE_N_ESTIMATORS,
        evals=[(dval, "val")],
        early_stopping_rounds=EARLY_STOP_ROUNDS,
        verbose_eval=False
    )

    val_proba = predict_proba_booster(booster, dval)
    val_pred  = np.argmax(val_proba, axis=1)
    acc = accuracy_score(y_val, val_pred)
    f1m = f1_score(y_val, val_pred, average="macro")

    best_it = getattr(booster, "best_iteration", None)
    if best_it is None:
        bntl = getattr(booster, "best_ntree_limit", BASE_N_ESTIMATORS)
        best_it = int(bntl) - 1

    search_rows.append({**p, "best_iteration": int(best_it), "val_accuracy": float(acc), "val_f1_macro": float(f1m)})
    if acc > best["acc"]:
        best = {"acc": acc, "params": p, "booster": booster, "best_round": int(best_it)}

pd.DataFrame(search_rows).sort_values(["val_accuracy","val_f1_macro"], ascending=False).to_csv(SEARCH_CSV, index=False)
best["booster"].save_model(MODEL_VAL_BEST)


val_proba = predict_proba_booster(best["booster"], dval)
val_summary = evaluate_with_reports(dval, y_val, val_proba, prefix="val_xgb_custom")


best_round_count = int(best["best_round"]) + 1  
final_params = make_params(**best["params"])
final_booster = xgb.train(
    final_params, dtrainval, num_boost_round=best_round_count,
    evals=[(dtest, "test")],  
    verbose_eval=False
)
final_booster.save_model(MODEL_FINAL)

test_proba = final_booster.predict(dtest)  
test_summary = evaluate_with_reports(dtest, y_test, test_proba, prefix="test_xgb_custom")


report = {
    "split_sizes": {"train": int(X_train.shape[0]), "val": int(X_val.shape[0]), "test": int(X_test.shape[0])},
    "val_search_best_params": best["params"],
    "val_best_round": int(best["best_round"]),
    "val_metrics": val_summary,
    "test_metrics": test_summary,
    "classes": class_names,
    "artifacts": {
        "search_csv": SEARCH_CSV,
        "scaler": SCALER_PATH,
        "val_model": MODEL_VAL_BEST,
        "final_model": MODEL_FINAL
    }
}
with open(REPORT_OUT, "w") as f:
    json.dump(report, f, indent=2, cls=NpEncoder)

print("\nSaved artifacts to:", OUT_DIR)
print("Best VAL acc:", f"{val_summary['accuracy']:.4f}")
print("Final TEST acc:", f"{test_summary['accuracy']:.4f}")
print("Search CSV:", SEARCH_CSV)
print("Scaler:", SCALER_PATH)
print("Val-best model:", MODEL_VAL_BEST)
print("Final model (train+val):", MODEL_FINAL)
print("Report JSON:", REPORT_OUT)



Saved artifacts to: D:\Research\Custom CNN\Without Augmented\XGB_Custom
Best VAL acc: 0.7545
Final TEST acc: 0.7198
Search CSV: D:\Research\Custom CNN\Without Augmented\XGB_Custom\xgb_custom_val_search_results.csv
Scaler: D:\Research\Custom CNN\Without Augmented\XGB_Custom\xgb_custom_scaler.joblib
Val-best model: D:\Research\Custom CNN\Without Augmented\XGB_Custom\xgb_custom_valbest.json
Final model (train+val): D:\Research\Custom CNN\Without Augmented\XGB_Custom\xgb_custom_final_trainval.json
Report JSON: D:\Research\Custom CNN\Without Augmented\XGB_Custom\xgb_custom_report.json


#### SVM    

In [ ]:
import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
import joblib

BASE_DIR   = r'D:\Research\Custom CNN\Without Augmented'
CSV_PATH   = r'D:\Research\Custom CNN\Without Augmented\features_256d_custom.csv'
OUT_DIR    = os.path.join(BASE_DIR, "SVM") 

os.makedirs(OUT_DIR, exist_ok=True)
REPORT_OUT = os.path.join(OUT_DIR, "svm_report.json")
MODEL_VAL_BEST = os.path.join(OUT_DIR, "svm_valbest.joblib")
MODEL_FINAL    = os.path.join(OUT_DIR, "svm_final_trainval.joblib")
SEARCH_CSV     = os.path.join(OUT_DIR, "svm_val_search_results.csv")


df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(np.int64)

class_map = df.sort_values("class_idx")[["class_idx","label"]].drop_duplicates()
class_names = class_map.set_index("class_idx")["label"].reindex(sorted(class_map["class_idx"])).tolist()
n_classes = len(np.unique(y))
rng = 42

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=rng
)


scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)


def make_svm(C, gamma):
    
    return SVC(kernel="rbf", C=C, gamma=gamma, probability=True, random_state=rng)

C_LIST     = np.logspace(-2, 3, 12)     
GAMMA_LIST = np.logspace(-5, 1, 13)    

rows, best = [], {"acc": -1, "params": None, "model": None}
for C in C_LIST:
    for g in GAMMA_LIST:
        model = make_svm(C, g)
        model.fit(X_train_s, y_train)
        yv_pred = model.predict(X_val_s)
        acc = accuracy_score(y_val, yv_pred)
        f1m = f1_score(y_val, yv_pred, average="macro")
        rows.append({"C": C, "gamma": g, "val_accuracy": acc, "val_f1_macro": f1m})
        if acc > best["acc"]:
            best.update({"acc": acc, "params": {"C": float(C), "gamma": float(g)}, "model": model})

pd.DataFrame(rows).sort_values(["val_accuracy","val_f1_macro"], ascending=False).to_csv(SEARCH_CSV, index=False)
joblib.dump({"scaler": scaler, "model": best["model"]}, MODEL_VAL_BEST)


def save_confmat_and_reports(Xs, y_true, model, prefix):
    y_proba = model.predict_proba(Xs)
    y_pred  = y_proba.argmax(1)

    acc   = accuracy_score(y_true, y_pred)
    f1m   = f1_score(y_true, y_pred, average="macro")
    f1w   = f1_score(y_true, y_pred, average="weighted")
    precm = precision_score(y_true, y_pred, average="macro")
    precw = precision_score(y_true, y_pred, average="weighted")
    recm  = recall_score(y_true, y_pred, average="macro")
    recw  = recall_score(y_true, y_pred, average="weighted")

    pd.DataFrame(classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
                 ).transpose().to_csv(os.path.join(OUT_DIR, f"{prefix}_classification_report.csv"))

    cm = confusion_matrix(y_true, y_pred)
    pd.DataFrame(cm, index=class_names, columns=class_names).to_csv(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.csv"))
    plt.figure(figsize=(6,5)); plt.imshow(cm, interpolation='nearest'); plt.title(f"Confusion Matrix — {prefix}")
    plt.colorbar(); ticks=np.arange(len(class_names))
    plt.xticks(ticks, class_names, rotation=45, ha="right"); plt.yticks(ticks, class_names)
    th=cm.max()/2
    for i in range(cm.shape[0]):
      for j in range(cm.shape[1]):
        plt.text(j,i,str(cm[i,j]),ha="center",va="center",color="white" if cm[i,j]>th else "black")
    plt.tight_layout(); plt.ylabel("True"); plt.xlabel("Pred")
    plt.savefig(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.png"), dpi=160, bbox_inches="tight"); plt.close()

    metrics_extra = {}
    if n_classes > 1:
        y_bin = label_binarize(y_true, classes=np.arange(n_classes))
        fpr, tpr, roc_auc, prec, rec, ap = {}, {}, {}, {}, {}, {}
        for c in range(n_classes):
            fpr[c], tpr[c], _ = roc_curve(y_bin[:, c], y_proba[:, c]); roc_auc[c] = auc(fpr[c], tpr[c])
            prec[c], rec[c], _ = precision_recall_curve(y_bin[:, c], y_proba[:, c]); ap[c] = average_precision_score(y_bin[:, c], y_proba[:, c])
        fpr["micro"], tpr["micro"], _ = roc_curve(y_bin.ravel(), y_proba.ravel()); roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        prec["micro"], rec["micro"], _ = precision_recall_curve(y_bin.ravel(), y_proba.ravel()); ap["micro"] = average_precision_score(y_bin, y_proba, average="micro")
        all_fpr = np.unique(np.concatenate([fpr[c] for c in range(n_classes)])); mean_tpr = np.zeros_like(all_fpr)
        for c in range(n_classes): mean_tpr += np.interp(all_fpr, fpr[c], tpr[c])
        mean_tpr /= n_classes; roc_auc["macro"] = auc(all_fpr, mean_tpr); ap["macro"] = np.mean([ap[c] for c in range(n_classes)])
        rows = []; 
        for key in list(range(n_classes)) + ["micro"]:
            for xi, yi in zip(fpr[key], tpr[key]): rows.append({"curve": f"ROC_{key}", "fpr": float(xi), "tpr": float(yi)})
        for xi, yi in zip(all_fpr, mean_tpr): rows.append({"curve": "ROC_macro", "fpr": float(xi), "tpr": float(yi)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_roc_points.csv"), index=False)
        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for pi, ri in zip(prec[key], rec[key]): rows.append({"curve": f"PR_{key}", "precision": float(pi), "recall": float(ri)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_pr_points.csv"), index=False)

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(fpr[c], tpr[c], lw=1.2, label=f"{class_names[c]} (AUC={roc_auc[c]:.3f})")
        plt.plot(fpr["micro"], tpr["micro"], lw=2, linestyle="--", label=f"micro (AUC={roc_auc['micro']:.3f})")
        plt.plot([0,1],[0,1],"k--", lw=1); plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {prefix}")
        plt.legend(loc="lower right", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.png"), dpi=160); plt.close()

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(rec[c], prec[c], lw=1.2, label=f"{class_names[c]} (AP={ap[c]:.3f})")
        plt.plot(rec["micro"], rec["micro"], lw=2, linestyle="--", label=f"micro (AP={ap['micro']:.3f})")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {prefix}")
        plt.legend(loc="lower left", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.png"), dpi=160); plt.close()

        metrics_extra = {
            "roc_auc_per_class": {class_names[c]: float(roc_auc[c]) for c in range(n_classes)},
            "roc_auc_micro": float(roc_auc["micro"]),
            "roc_auc_macro": float(roc_auc["macro"]),
            "ap_per_class": {class_names[c]: float(ap[c]) for c in range(n_classes)},
            "ap_micro": float(ap["micro"]),
            "ap_macro": float(ap["macro"]),
        }

    return {
        "accuracy": float(acc),
        "f1_macro": float(f1m),
        "f1_weighted": float(f1w),
        "precision_macro": float(precm),
        "precision_weighted": float(precw),
        "recall_macro": float(recm),
        "recall_weighted": float(recw),
        **metrics_extra
    }

val_summary = save_confmat_and_reports(X_val_s, y_val, best["model"], prefix="val_svm")
joblib.dump({"scaler": scaler, "model": best["model"]}, MODEL_VAL_BEST)

p = best["params"]
final_model = make_svm(p["C"], p["gamma"])
final_model.fit(np.vstack([X_train_s, X_val_s]), np.concatenate([y_train, y_val]))
joblib.dump({"scaler": scaler, "model": final_model}, MODEL_FINAL)

test_summary = save_confmat_and_reports(X_test_s, y_test, final_model, prefix="test_svm")

report = {
    "split_sizes": {"train": int(X_train.shape[0]), "val": int(X_val.shape[0]), "test": int(X_test.shape[0])},
    "val_search_best_params": p,
    "val_metrics": val_summary,
    "test_metrics": test_summary,
    "classes": class_names
}
with open(REPORT_OUT, "w") as f:
    json.dump(report, f, indent=2)

print("\nSaved artifacts to:", OUT_DIR)
print("Best VAL acc:", f"{val_summary['accuracy']:.4f}")
print("Final TEST acc:", f"{test_summary['accuracy']:.4f}")
print("Report:", REPORT_OUT)


Saved artifacts to: D:\Research\Custom CNN\Without Augmented\SVM
Best VAL acc: 0.8136
Final TEST acc: 0.7813
Report: D:\Research\Custom CNN\Without Augmented\SVM\svm_report.json


#### KNN

In [ ]:
import os, json, joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product

import matplotlib as mpl
mpl.rcParams['svg.fonttype'] = 'none'  

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
from sklearn.neighbors import KNeighborsClassifier


BASE_DIR   = r'D:\Research\Custom CNN\Without Augmented'
CSV_PATH   = r'D:\Research\Custom CNN\Without Augmented\features_256d_custom.csv'
OUT_DIR    = os.path.join(BASE_DIR, "KNN")  
os.makedirs(OUT_DIR, exist_ok=True)
rng = 42

SEARCH_CSV     = os.path.join(OUT_DIR, "knn_val_search_results.csv")
MODEL_VALBEST  = os.path.join(OUT_DIR, "knn_valbest.joblib")
MODEL_FINAL    = os.path.join(OUT_DIR, "knn_final_trainval.joblib")
SCALER_PATH    = os.path.join(OUT_DIR, "knn_scaler.joblib")
REPORT_JSON    = os.path.join(OUT_DIR, "knn_report.json")


df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
if not feat_cols:
    raise RuntimeError("No feature columns found like f0..f255")
X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(np.int64)

if "label" in df.columns:
    class_map = df.sort_values("class_idx")[["class_idx","label"]].drop_duplicates()
    class_names = (
        class_map.set_index("class_idx")["label"]
        .reindex(sorted(class_map["class_idx"]))
        .tolist()
    )
else:
    class_names = [str(i) for i in sorted(np.unique(y))]
n_classes = len(np.unique(y))

print(f"[KNN] Found feature columns: {len(feat_cols)}")
print("[KNN] First 10 feature cols:", feat_cols[:10])
print("[KNN] X shape:", X.shape)


X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=rng
)


scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)
joblib.dump(scaler, SCALER_PATH)

def save_confmat_and_curves(y_true, proba, y_pred, prefix):
    
    rep = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    pd.DataFrame(rep).transpose().to_csv(os.path.join(OUT_DIR, f"{prefix}_classification_report.csv"))

    
    cm = confusion_matrix(y_true, y_pred)
    pd.DataFrame(cm, index=class_names, columns=class_names)\
      .to_csv(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.csv"))

    plt.figure(figsize=(6,5))
    plt.imshow(cm, interpolation='nearest')
    plt.title(f"Confusion Matrix — {prefix}")
    plt.colorbar()
    ticks = np.arange(len(class_names))
    plt.xticks(ticks, class_names, rotation=45, ha="right")
    plt.yticks(ticks, class_names)
    th = cm.max()/2 if cm.max() > 0 else 0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     ha="center", va="center",
                     color="white" if cm[i, j] > th else "black")
    plt.tight_layout(); plt.ylabel("True"); plt.xlabel("Pred")
    plt.savefig(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.png"), dpi=160, bbox_inches="tight")
    plt.savefig(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.svg"), bbox_inches="tight")
    plt.close()

    
    metrics_extra = {}
    if proba is not None and n_classes > 1:
        y_bin = label_binarize(y_true, classes=np.arange(n_classes))
        proba = np.asarray(proba)
        fpr, tpr, roc_auc = {}, {}, {}
        prec, rec, ap = {}, {}, {}

        for c in range(n_classes):
            fpr[c], tpr[c], _ = roc_curve(y_bin[:, c], proba[:, c])
            roc_auc[c] = auc(fpr[c], tpr[c])
            prec[c], rec[c], _ = precision_recall_curve(y_bin[:, c], proba[:, c])
            ap[c] = average_precision_score(y_bin[:, c], proba[:, c])

        
        plt.figure(figsize=(7,6))
        for c in range(n_classes):
            plt.plot(fpr[c], tpr[c], lw=1.2, label=f"{class_names[c]} (AUC={roc_auc[c]:.3f})")
        plt.plot([0,1],[0,1],"k--", lw=1)
        plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {prefix}")
        plt.legend(loc="lower right", fontsize=8)
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.png"), dpi=160)
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.svg"))
        plt.close()

        
        plt.figure(figsize=(7,6))
        for c in range(n_classes):
            plt.plot(rec[c], prec[c], lw=1.2, label=f"{class_names[c]} (AP={ap[c]:.3f})")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {prefix}")
        plt.legend(loc="lower left", fontsize=8)
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.png"), dpi=160)
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.svg"))
        plt.close()

        metrics_extra.update({
            "roc_auc_per_class": {class_names[c]: float(roc_auc[c]) for c in range(n_classes)},
            "ap_per_class": {class_names[c]: float(ap[c]) for c in range(n_classes)},
            "roc_auc_macro": float(np.mean(list(roc_auc.values()))),
            "ap_macro": float(np.mean(list(ap.values()))),
        })

    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average="macro")
    f1w = f1_score(y_true, y_pred, average="weighted")
    precm = precision_score(y_true, y_pred, average="macro")
    precw = precision_score(y_true, y_pred, average="weighted")
    recm = recall_score(y_true, y_pred, average="macro")
    recw = recall_score(y_true, y_pred, average="weighted")

    return {
        "accuracy": float(acc),
        "f1_macro": float(f1m),
        "f1_weighted": float(f1w),
        "precision_macro": float(precm),
        "precision_weighted": float(precw),
        "recall_macro": float(recm),
        "recall_weighted": float(recw),
        **metrics_extra
    }

grid = {
    "n_neighbors": [3, 5, 9, 15],
    "metric":      ["euclidean", "cosine"],
    "weights":     ["distance"],   
    "leaf_size":   [30],
    "p":           [2],            
}

def knn_from(p):
    return KNeighborsClassifier(
        n_neighbors=p["n_neighbors"],
        metric=p["metric"],
        weights=p["weights"],
        leaf_size=p["leaf_size"],
        p=p["p"],
        algorithm="auto",          
        n_jobs=-1
    )

keys = list(grid.keys())
search_rows = []
best = {"acc": -1.0, "params": None, "model": None}

for vals in product(*[grid[k] for k in keys]):
    p = dict(zip(keys, vals))
    model = knn_from(p).fit(X_train_s, y_train)
    yv = model.predict(X_val_s)
    acc = accuracy_score(y_val, yv)
    f1m = f1_score(y_val, yv, average="macro")
    search_rows.append({**p, "val_accuracy": float(acc), "val_f1_macro": float(f1m)})
    if acc > best["acc"]:
        best = {"acc": acc, "params": p, "model": model}

pd.DataFrame(search_rows).sort_values(
    ["val_accuracy","val_f1_macro"], ascending=False
).to_csv(SEARCH_CSV, index=False)
joblib.dump({"scaler": scaler, "model": best["model"]}, MODEL_VALBEST)


yv_pred = best["model"].predict(X_val_s)
yv_prob = best["model"].predict_proba(X_val_s) if hasattr(best["model"], "predict_proba") else None
val_summary = save_confmat_and_curves(y_val, yv_prob, yv_pred, "val_knn_custom")

X_trv = np.vstack([X_train_s, X_val_s]); y_trv = np.concatenate([y_train, y_val])
final_model = knn_from(best["params"]).fit(X_trv, y_trv)
joblib.dump({"scaler": scaler, "model": final_model}, MODEL_FINAL)

yt_pred = final_model.predict(X_test_s)
yt_prob = final_model.predict_proba(X_test_s) if hasattr(final_model, "predict_proba") else None
test_summary = save_confmat_and_curves(y_test, yt_prob, yt_pred, "test_knn_custom")

with open(REPORT_JSON, "w") as f:
    json.dump({
        "split_sizes": {
            "train": int(X_train.shape[0]),
            "val":   int(X_val.shape[0]),
            "test":  int(X_test.shape[0])
        },
        "val_search_best_params": best["params"],
        "val_metrics": val_summary,
        "test_metrics": test_summary,
        "classes": class_names,
        "features_used": feat_cols,
        "artifacts": {
            "search_csv": SEARCH_CSV,
            "scaler": SCALER_PATH,
            "val_model": MODEL_VALBEST,
            "final_model": MODEL_FINAL
        }
    }, f, indent=2)

print("\n[KNN] Best VAL acc:", f"{best['acc']:.4f}")
print("[KNN] Final TEST acc:", f"{test_summary['accuracy']:.4f}")
print("[KNN] Saved to:", OUT_DIR)


[KNN] Found feature columns: 256
[KNN] First 10 feature cols: ['f000', 'f001', 'f002', 'f003', 'f004', 'f005', 'f006', 'f007', 'f008', 'f009']
[KNN] X shape: (2195, 256)

[KNN] Best VAL acc: 0.7273
[KNN] Final TEST acc: 0.6834
[KNN] Saved to: D:\Research\Custom CNN\Without Augmented\KNN


#### RF

In [ ]:
import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
from sklearn.ensemble import RandomForestClassifier
import joblib


BASE_DIR   = r'D:\Research\Custom CNN\Without Augmented'
CSV_PATH   = r'D:\Research\Custom CNN\Without Augmented\features_256d_custom.csv'
OUT_DIR    = os.path.join(BASE_DIR, "RF") 

os.makedirs(OUT_DIR, exist_ok=True)
REPORT_OUT = os.path.join(OUT_DIR, "rf_report.json")
MODEL_VAL_BEST = os.path.join(OUT_DIR, "rf_valbest.joblib")
MODEL_FINAL    = os.path.join(OUT_DIR, "rf_final_trainval.joblib")
SEARCH_CSV     = os.path.join(OUT_DIR, "rf_val_search_results.csv")


df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(np.int64)

class_map = df.sort_values("class_idx")[["class_idx","label"]].drop_duplicates()
class_names = class_map.set_index("class_idx")["label"].reindex(sorted(class_map["class_idx"])).tolist()
n_classes = len(np.unique(y))
rng = 42


X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=rng
)

def make_rf(params):
    return RandomForestClassifier(
        random_state=rng, n_jobs=-1, oob_score=False, **params
    )

GRID = {
    "n_estimators":      [300, 600, 1000],
    "max_depth":         [None, 12, 20],
    "max_features":      ["sqrt", "log2", 0.5],
    "min_samples_leaf":  [1, 2, 4],
    "min_samples_split": [2, 4, 8],
    "bootstrap":         [True],
}
keys = list(GRID.keys())

search_rows, best = [], {"acc": -1, "params": None, "model": None}
for values in product(*[GRID[k] for k in keys]):
    p = dict(zip(keys, values))
    mdl = make_rf(p)
    mdl.fit(X_train, y_train)
    yv_pred = mdl.predict(X_val)
    acc = accuracy_score(y_val, yv_pred)
    f1m = f1_score(y_val, yv_pred, average="macro")
    search_rows.append({**p, "val_accuracy": acc, "val_f1_macro": f1m})
    if acc > best["acc"]:
        best = {"acc": acc, "params": p, "model": mdl}

pd.DataFrame(search_rows).sort_values(["val_accuracy","val_f1_macro"], ascending=False)\
    .to_csv(SEARCH_CSV, index=False)
joblib.dump(best["model"], MODEL_VAL_BEST)


def save_confmat_and_reports(Xs, y_true, model, prefix):
    y_proba = model.predict_proba(Xs)
    y_pred  = y_proba.argmax(1)

    acc   = accuracy_score(y_true, y_pred)
    f1m   = f1_score(y_true, y_pred, average="macro")
    f1w   = f1_score(y_true, y_pred, average="weighted")
    precm = precision_score(y_true, y_pred, average="macro")
    precw = precision_score(y_true, y_pred, average="weighted")
    recm  = recall_score(y_true, y_pred, average="macro")
    recw  = recall_score(y_true, y_pred, average="weighted")

    pd.DataFrame(classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
                 ).transpose().to_csv(os.path.join(OUT_DIR, f"{prefix}_classification_report.csv"))
    cm = confusion_matrix(y_true, y_pred)
    pd.DataFrame(cm, index=class_names, columns=class_names)\
        .to_csv(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.csv"))

    metrics_extra = {}
    if n_classes > 1:
        y_bin = label_binarize(y_true, classes=np.arange(n_classes))
        fpr, tpr, roc_auc, prec, rec, ap = {}, {}, {}, {}, {}, {}
        for c in range(n_classes):
            fpr[c], tpr[c], _ = roc_curve(y_bin[:, c], y_proba[:, c]); roc_auc[c] = auc(fpr[c], tpr[c])
            prec[c], rec[c], _ = precision_recall_curve(y_bin[:, c], y_proba[:, c]); ap[c] = average_precision_score(y_bin[:, c], y_proba[:, c])
        fpr["micro"], tpr["micro"], _ = roc_curve(y_bin.ravel(), y_proba.ravel()); roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        prec["micro"], rec["micro"], _ = precision_recall_curve(y_bin.ravel(), y_proba.ravel()); ap["micro"] = average_precision_score(y_bin, y_proba, average="micro")
        all_fpr = np.unique(np.concatenate([fpr[c] for c in range(n_classes)])); mean_tpr = np.zeros_like(all_fpr)
        for c in range(n_classes): mean_tpr += np.interp(all_fpr, fpr[c], tpr[c])
        mean_tpr /= n_classes; roc_auc["macro"] = auc(all_fpr, mean_tpr); ap["macro"] = np.mean([ap[c] for c in range(n_classes)])

        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for xi, yi in zip(fpr[key], tpr[key]):
                rows.append({"curve": f"ROC_{key}", "fpr": float(xi), "tpr": float(yi)})
        for xi, yi in zip(all_fpr, mean_tpr):
            rows.append({"curve": "ROC_macro", "fpr": float(xi), "tpr": float(yi)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_roc_points.csv"), index=False)

        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for pi, ri in zip(prec[key], rec[key]):
                rows.append({"curve": f"PR_{key}", "precision": float(pi), "recall": float(ri)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_pr_points.csv"), index=False)

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(fpr[c], tpr[c], lw=1.2, label=f"{class_names[c]} (AUC={roc_auc[c]:.3f})")
        plt.plot(fpr["micro"], tpr["micro"], lw=2, linestyle="--", label=f"micro (AUC={roc_auc['micro']:.3f})")
        plt.plot([0,1],[0,1],"k--", lw=1); plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {prefix}")
        plt.legend(loc="lower right", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.png"), dpi=160); plt.close()

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(rec[c], prec[c], lw=1.2, label=f"{class_names[c]} (AP={ap[c]:.3f})")
        plt.plot(rec["micro"], rec["micro"], lw=2, linestyle="--", label=f"micro (AP={ap['micro']:.3f})")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {prefix}")
        plt.legend(loc="lower left", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.png"), dpi=160); plt.close()

        metrics_extra = {
            "roc_auc_per_class": {class_names[c]: float(roc_auc[c]) for c in range(n_classes)},
            "roc_auc_micro": float(roc_auc["micro"]),
            "roc_auc_macro": float(roc_auc["macro"]),
            "ap_per_class": {class_names[c]: float(ap[c]) for c in range(n_classes)},
            "ap_micro": float(ap["micro"]),
            "ap_macro": float(ap["macro"]),
        }

    return {
        "accuracy": float(acc),
        "f1_macro": float(f1m),
        "f1_weighted": float(f1w),
        "precision_macro": float(precm),
        "precision_weighted": float(precw),
        "recall_macro": float(recm),
        "recall_weighted": float(recw),
        **metrics_extra
    }

val_summary = save_confmat_and_reports(X_val, y_val, best["model"], prefix="val_rf")
joblib.dump(best["model"], MODEL_VAL_BEST)


p = best["params"].copy()
final_model = make_rf(p)
final_model.fit(np.vstack([X_train, X_val]), np.concatenate([y_train, y_val]))
joblib.dump(final_model, MODEL_FINAL)

test_summary = save_confmat_and_reports(X_test, y_test, final_model, prefix="test_rf")

report = {
    "split_sizes": {"train": int(X_train.shape[0]), "val": int(X_val.shape[0]), "test": int(X_test.shape[0])},
    "val_search_best_params": p,
    "val_metrics": val_summary,
    "test_metrics": test_summary,
    "classes": class_names
}
with open(REPORT_OUT, "w") as f:
    json.dump(report, f, indent=2)

print("\nSaved artifacts to:", OUT_DIR)
print("Best VAL acc:", f"{val_summary['accuracy']:.4f}")
print("Final TEST acc:", f"{test_summary['accuracy']:.4f}")
print("Report:", REPORT_OUT)
print("Val-best model:", MODEL_VAL_BEST)
print("Final model (train+val):", MODEL_FINAL)
print("Search table:", SEARCH_CSV)


Saved artifacts to: D:\Research\Custom CNN\Without Augmented\RF
Best VAL acc: 0.7091
Final TEST acc: 0.7267
Report: D:\Research\Custom CNN\Without Augmented\RF\rf_report.json
Val-best model: D:\Research\Custom CNN\Without Augmented\RF\rf_valbest.joblib
Final model (train+val): D:\Research\Custom CNN\Without Augmented\RF\rf_final_trainval.joblib
Search table: D:\Research\Custom CNN\Without Augmented\RF\rf_val_search_results.csv


#### CatBoost

In [ ]:

import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
from catboost import CatBoostClassifier
import joblib

CSV_PATH   = r"D:\Research\Custom CNN\Without Augmented\features_256d_custom.csv"   
OUT_DIR    = "/kaggle/working/Customized CNN/CAT"
os.makedirs(OUT_DIR, exist_ok=True)
REPORT_OUT = os.path.join(OUT_DIR, "cat_report.json")
MODEL_VAL_BEST = os.path.join(OUT_DIR, "cat_valbest.cbm")
MODEL_FINAL    = os.path.join(OUT_DIR, "cat_final_trainval.cbm")
SEARCH_CSV     = os.path.join(OUT_DIR, "cat_val_search_results.csv")

df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
X = df[feat_cols].values.astype(np.float32)
y = df["class_idx"].values.astype(np.int64)

class_map = df.sort_values("class_idx")[["class_idx","label"]].drop_duplicates()
class_names = class_map.set_index("class_idx")["label"].reindex(sorted(class_map["class_idx"])).tolist()
n_classes = len(np.unique(y))
rng = 42


X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.125, stratify=y_trainval, random_state=rng
)

scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

def make_cat(params):
    return CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="MultiClass",
        random_seed=rng,
        allow_writing_files=False,
        
        task_type="GPU" if os.environ.get("NVIDIA_VISIBLE_DEVICES") not in (None, "", "none") else "CPU",
        **params
    )

GRID = {
    "iterations":       [2000],          
    "learning_rate":    [0.03, 0.06],
    "depth":            [4, 5, 6],
    "l2_leaf_reg":      [1.0, 3.0, 5.0],
    "border_count":     [128],           
    "random_strength":  [1.0, 2.0],      
    "bagging_temperature": [0.0, 1.0],   
    "grow_policy":      ["SymmetricTree"],  
}

search_rows, best = [], {"acc": -1, "params": None, "model": None}
keys = list(GRID.keys())
for values in product(*[GRID[k] for k in keys]):
    p = dict(zip(keys, values))
    model = make_cat(p)
    model.fit(
        X_train_s, y_train,
        eval_set=(X_val_s, y_val),
        use_best_model=True,
        early_stopping_rounds=50,
        verbose=False
    )
    yv_pred = model.predict(X_val_s).astype(int).ravel()
    acc = accuracy_score(y_val, yv_pred)
    f1m = f1_score(y_val, yv_pred, average="macro")
    row = {**p,
           "best_iteration": int(model.get_best_iteration()),
           "val_accuracy": acc,
           "val_f1_macro": f1m}
    search_rows.append(row)
    if acc > best["acc"]:
        best = {"acc": acc, "params": p, "model": model}

pd.DataFrame(search_rows).sort_values(["val_accuracy","val_f1_macro"], ascending=False)\
    .to_csv(SEARCH_CSV, index=False)
best["model"].save_model(MODEL_VAL_BEST)

def save_confmat_and_reports(Xs, y_true, model, prefix):
    y_proba = model.predict_proba(Xs)
    y_pred  = y_proba.argmax(1)

    acc   = accuracy_score(y_true, y_pred)
    f1m   = f1_score(y_true, y_pred, average="macro")
    f1w   = f1_score(y_true, y_pred, average="weighted")
    precm = precision_score(y_true, y_pred, average="macro")
    precw = precision_score(y_true, y_pred, average="weighted")
    recm  = recall_score(y_true, y_pred, average="macro")
    recw  = recall_score(y_true, y_pred, average="weighted")

    pd.DataFrame(
        classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    ).transpose().to_csv(os.path.join(OUT_DIR, f"{prefix}_classification_report.csv"))

    cm = confusion_matrix(y_true, y_pred)
    pd.DataFrame(cm, index=class_names, columns=class_names)\
        .to_csv(os.path.join(OUT_DIR, f"{prefix}_confusion_matrix.csv"))

    metrics_extra = {}
    if n_classes > 1:
        y_bin = label_binarize(y_true, classes=np.arange(n_classes))
        fpr, tpr, roc_auc, prec, rec, ap = {}, {}, {}, {}, {}, {}
        for c in range(n_classes):
            fpr[c], tpr[c], _ = roc_curve(y_bin[:, c], y_proba[:, c]); roc_auc[c] = auc(fpr[c], tpr[c])
            prec[c], rec[c], _ = precision_recall_curve(y_bin[:, c], y_proba[:, c]); ap[c] = average_precision_score(y_bin[:, c], y_proba[:, c])
        fpr["micro"], tpr["micro"], _ = roc_curve(y_bin.ravel(), y_proba.ravel()); roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        prec["micro"], rec["micro"], _ = precision_recall_curve(y_bin.ravel(), y_proba.ravel()); ap["micro"] = average_precision_score(y_bin, y_proba, average="micro")
        all_fpr = np.unique(np.concatenate([fpr[c] for c in range(n_classes)])); mean_tpr = np.zeros_like(all_fpr)
        for c in range(n_classes): mean_tpr += np.interp(all_fpr, fpr[c], tpr[c])
        mean_tpr /= n_classes; roc_auc["macro"] = auc(all_fpr, mean_tpr); ap["macro"] = np.mean([ap[c] for c in range(n_classes)])

        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for xi, yi in zip(fpr[key], tpr[key]):
                rows.append({"curve": f"ROC_{key}", "fpr": float(xi), "tpr": float(yi)})
        for xi, yi in zip(all_fpr, mean_tpr):
            rows.append({"curve": "ROC_macro", "fpr": float(xi), "tpr": float(yi)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_roc_points.csv"), index=False)

        rows = []
        for key in list(range(n_classes)) + ["micro"]:
            for pi, ri in zip(prec[key], rec[key]):
                rows.append({"curve": f"PR_{key}", "precision": float(pi), "recall": float(ri)})
        pd.DataFrame(rows).to_csv(os.path.join(OUT_DIR, f"{prefix}_pr_points.csv"), index=False)

        # Plots
        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(fpr[c], tpr[c], lw=1.2, label=f"{class_names[c]} (AUC={roc_auc[c]:.3f})")
        plt.plot(fpr["micro"], tpr["micro"], lw=2, linestyle="--", label=f"micro (AUC={roc_auc['micro']:.3f})")
        plt.plot([0,1],[0,1],"k--", lw=1); plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {prefix}")
        plt.legend(loc="lower right", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_roc_curves.png"), dpi=160); plt.close()

        plt.figure(figsize=(7,6))
        for c in range(n_classes): plt.plot(rec[c], prec[c], lw=1.2, label=f"{class_names[c]} (AP={ap[c]:.3f})")
        plt.plot(rec["micro"], prec["micro"], lw=2, linestyle="--", label=f"micro (AP={ap['micro']:.3f})")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {prefix}")
        plt.legend(loc="lower left", fontsize=8); plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR, f"{prefix}_pr_curves.png"), dpi=160); plt.close()

        metrics_extra = {
            "roc_auc_per_class": {class_names[c]: float(roc_auc[c]) for c in range(n_classes)},
            "roc_auc_micro": float(roc_auc["micro"]),
            "roc_auc_macro": float(roc_auc["macro"]),
            "ap_per_class": {class_names[c]: float(ap[c]) for c in range(n_classes)},
            "ap_micro": float(ap["micro"]),
            "ap_macro": float(ap["macro"]),
        }

    return {
        "accuracy": float(acc),
        "f1_macro": float(f1m),
        "f1_weighted": float(f1w),
        "precision_macro": float(precm),
        "precision_weighted": float(precw),
        "recall_macro": float(recm),
        "recall_weighted": float(recw),
        **metrics_extra
    }

val_summary = save_confmat_and_reports(X_val_s, y_val, best["model"], prefix="val_cat")
best["model"].save_model(MODEL_VAL_BEST)

p = best["params"].copy()
final_model = make_cat(p)
final_model.fit(
    scaler.transform(np.vstack([X_train, X_val])),
    np.concatenate([y_train, y_val]),
    eval_set=(X_val_s, y_val),
    use_best_model=True,
    early_stopping_rounds=50,
    verbose=False
)
final_model.save_model(MODEL_FINAL)

test_summary = save_confmat_and_reports(X_test_s, y_test, final_model, prefix="test_cat")

report = {
    "split_sizes": {"train": int(X_train.shape[0]), "val": int(X_val.shape[0]), "test": int(X_test.shape[0])},
    "val_search_best_params": p,
    "val_metrics": val_summary,
    "test_metrics": test_summary,
    "classes": class_names
}
with open(REPORT_OUT, "w") as f:
    json.dump(report, f, indent=2)

joblib.dump(scaler, os.path.join(OUT_DIR, "standard_scaler.joblib"))

print("\nSaved artifacts to:", OUT_DIR)
print("Best VAL acc:", f"{val_summary['accuracy']:.4f}")
print("Final TEST acc:", f"{test_summary['accuracy']:.4f}")
print("Report:", REPORT_OUT)
print("Val-best model:", MODEL_VAL_BEST)
print("Final model (train+val):", MODEL_FINAL)
print("Search table:", SEARCH_CSV)



Saved artifacts to: /kaggle/working/Customized CNN/CAT
Best VAL acc: 0.7455
Final TEST acc: 0.7335
Report: /kaggle/working/Customized CNN/CAT\cat_report.json
Val-best model: /kaggle/working/Customized CNN/CAT\cat_valbest.cbm
Final model (train+val): /kaggle/working/Customized CNN/CAT\cat_final_trainval.cbm
Search table: /kaggle/working/Customized CNN/CAT\cat_val_search_results.csv


#### Ensamble

In [ ]:
import os, json, joblib, warnings
import numpy as np
import pandas as pd
from typing import Dict, Tuple

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, confusion_matrix, log_loss,
    roc_curve, auc, precision_recall_curve
)
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.linear_model import SGDClassifier, LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
rng = 42
np.random.seed(rng)


BASE_DIR  = r'D:\Research\Custom CNN\Without Augmented'
CSV_PATH  = os.path.join(BASE_DIR, 'features_256d_custom.csv')  
OUT_DIR   = os.path.join(BASE_DIR, 'Ensemble_256D')
os.makedirs(OUT_DIR, exist_ok=True)

MODEL_PATHS = {
    "rf":  os.path.join(BASE_DIR, r"RF\rf_final_trainval.joblib"),
    "svm": os.path.join(BASE_DIR, r"SVM\svm_final_trainval.joblib"),
    "xgb": os.path.join(BASE_DIR, r"XGB\xgb_final_trainval.joblib"),
    "lr":  os.path.join(BASE_DIR, r"LR\logreg_sgd_final_trainval.joblib"),
    "knn": os.path.join(BASE_DIR, r"KNN\knn_final_trainval.joblib"),
    
    "cat": os.path.join(BASE_DIR, r"CAT\cat_final_trainval.cbm"),
}
REPORT_JSON = os.path.join(OUT_DIR, "winner_report_256d.json")

df = pd.read_csv(CSV_PATH)
feat_cols = [c for c in df.columns if c.startswith("f") and c[1:].isdigit()]
if not feat_cols:
    raise RuntimeError("No 256-D feature columns like f0..f255 found.")
feat_cols = sorted(feat_cols, key=lambda c: int(c[1:]))

X_all = df[feat_cols].values.astype(np.float32)
y_all = df["class_idx"].values.astype(int)

classes = (
    df.sort_values("class_idx")[["class_idx","label"]]
      .drop_duplicates().sort_values("class_idx")["label"].tolist()
    if "label" in df.columns else [str(i) for i in sorted(df["class_idx"].unique())]
)
n_classes = len(np.unique(y_all))

X_tmp, X_test, y_tmp, y_test = train_test_split(
    X_all, y_all, test_size=0.20, stratify=y_all, random_state=rng
)
X_train, X_val, y_train, y_val = train_test_split(
    X_tmp, y_tmp, test_size=0.125, stratify=y_tmp, random_state=rng
)
print({"train": len(y_train), "val": len(y_val), "test": len(y_test)})


def _row_norm(p: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    p = np.nan_to_num(p, nan=0.0, posinf=0.0, neginf=0.0)
    p[p < 0] = 0.0
    s = p.sum(axis=1, keepdims=True); s[s <= 0] = 1.0
    p = p / s
    p = np.clip(p, eps, 1.0)
    p = p / p.sum(axis=1, keepdims=True)
    return p

def safe_log_probs(P: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    return np.log(_row_norm(P, eps))

def safe_softmax(L: np.ndarray, T: float = 1.0) -> np.ndarray:
    L = np.nan_to_num(L, nan=0.0, posinf=0.0, neginf=0.0) / max(T, 1e-6)
    L = L - np.max(L, axis=1, keepdims=True)
    E = np.exp(np.clip(L, -700, 700))
    return _row_norm(E)

def entropy_and_margin(P: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    p = _row_norm(P)
    ent = -(p * np.log(p)).sum(axis=1, keepdims=True)
    top2 = np.partition(p, -2, axis=1)[:, -2:]
    mar = (top2[:, 1] - top2[:, 0]).reshape(-1, 1)
    return ent, mar

def cm_plot(cm, classes, title, out_png):
    plt.figure(figsize=(6,5))
    plt.imshow(cm, interpolation='nearest')
    plt.title(title)
    plt.colorbar()
    t = np.arange(len(classes))
    plt.xticks(t, classes, rotation=45, ha="right"); plt.yticks(t, classes)
    th = cm.max()/2 if cm.max() > 0 else 0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, str(cm[i,j]), ha="center", va="center",
                     color="white" if cm[i,j] > th else "black")
    plt.tight_layout(); plt.ylabel("True"); plt.xlabel("Pred")
    plt.savefig(out_png, dpi=160, bbox_inches="tight"); plt.close()


def load_bundle_safe(name: str, path: str):
    try:
        if name == "cat" and path.lower().endswith(".cbm"):
            try:
                from catboost import CatBoostClassifier
                m = CatBoostClassifier()
                m.load_model(path)
                return m, None
            except Exception as e:
                print(f"[WARN] CatBoost load failed: {e}")
                return None, None
        obj = joblib.load(path)
        if isinstance(obj, dict) and "model" in obj:
            return obj["model"], obj.get("scaler", None)
        return obj, None
    except Exception as e:
        print(f"[WARN] Load failed for {name} @ {path}: {e}")
        return None, None

def predict_logits(model, X) -> np.ndarray:
    if hasattr(model, "decision_function"):
        d = np.asarray(model.decision_function(X))
        if d.ndim == 1:
            d = np.vstack([-d, d]).T
        return np.nan_to_num(d, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float64)
    elif hasattr(model, "predict_proba"):
        return safe_log_probs(np.asarray(model.predict_proba(X), dtype=np.float64))
    else:
        pred = model.predict(X)
        L = np.full((X.shape[0], n_classes), -10.0, dtype=np.float64)
        L[np.arange(X.shape[0]), pred] = 10.0
        return L

def predict_proba_safe(model, X) -> np.ndarray:
    if hasattr(model, "predict_proba"):
        p = model.predict_proba(X)
        if p.ndim == 1:  # binary fallback
            p = np.vstack([1-p, p]).T
        return _row_norm(np.asarray(p, dtype=np.float64))
    else:
        return safe_softmax(predict_logits(model, X), 1.0)

def tta_probs(model, X, n=6, seed=42):
    rng_local = np.random.default_rng(seed)
    Ps = []
    for _ in range(n):
        X_aug = X * (1.0 + rng_local.normal(0, 0.01, X.shape)) + rng_local.normal(0, 0.005, X.shape)
        Ps.append(_row_norm(predict_proba_safe(model, X_aug)))
    return _row_norm(np.mean(Ps, axis=0))


loaded_models: Dict[str, object] = {}
loaded_scalers: Dict[str, object] = {}
for name, path in MODEL_PATHS.items():
    m, s = load_bundle_safe(name, path)
    if m is not None:
        loaded_models[name] = m
        loaded_scalers[name] = s
print("Loaded bases:", list(loaded_models.keys()))

K = 5
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=rng)

extra_fold_bases = [
    ("rf_in",  "std"),
    ("lr_in",  "std"),
    ("et_in",  "std"),
    ("knn5_euc_in", "std"),
    ("ridge_in", "std"),
    ("linsvc_platt_in", "std"),
    ("lda_shrink_in", "std"),
]
all_base_names = sorted(set(list(loaded_models.keys()) + [b for b,_ in extra_fold_bases]))
print("All base names:", all_base_names)

oof_logits = {b: np.zeros((len(y_train), n_classes), dtype=np.float64) for b in all_base_names}
oof_probs  = {b: np.zeros((len(y_train), n_classes), dtype=np.float64) for b in all_base_names}
val_logits = {b: np.zeros((len(y_val),   n_classes), dtype=np.float64) for b in all_base_names}
val_probs  = {b: np.zeros((len(y_val),   n_classes), dtype=np.float64) for b in all_base_names}
test_logits= {b: np.zeros((len(y_test),  n_classes), dtype=np.float64) for b in all_base_names}
test_probs = {b: np.zeros((len(y_test),  n_classes), dtype=np.float64) for b in all_base_names}

std_full = StandardScaler().fit(X_train)
bag_counts = {b: 0 for b,_ in extra_fold_bases}

def build_fold_model(tag: str):
    if tag == "rf_in":
        return RandomForestClassifier(n_estimators=600, max_features="sqrt", bootstrap=True, random_state=rng, n_jobs=-1)
    if tag == "et_in":
        return ExtraTreesClassifier(n_estimators=800, max_features="sqrt", bootstrap=False, random_state=rng, n_jobs=-1)
    if tag == "lr_in":
        return SGDClassifier(loss="log_loss", penalty="l2", alpha=1e-4, learning_rate="optimal",
                             max_iter=3500, tol=1e-4, random_state=rng, n_jobs=-1)
    if tag == "knn5_euc_in":
        return KNeighborsClassifier(n_neighbors=5, metric="euclidean", weights="distance", n_jobs=-1)
    if tag == "ridge_in":
        return RidgeClassifier(alpha=1.0, random_state=rng)
    if tag == "linsvc_platt_in":
        base = LinearSVC(C=1.0, random_state=rng)
        return CalibratedClassifierCV(estimator=base, method="sigmoid", cv=3)
    if tag == "lda_shrink_in":
        return LinearDiscriminantAnalysis(solver="lsqr", shrinkage="auto")
    raise ValueError(tag)

for fold, (tr_idx, oof_idx) in enumerate(skf.split(X_train, y_train), 1):
    Xtr, Xoo = X_train[tr_idx], X_train[oof_idx]
    ytr, yoo = y_train[tr_idx], y_train[oof_idx]

    Xtr_s, Xoo_s = std_full.transform(Xtr), std_full.transform(Xoo)
    Xva_s, Xte_s = std_full.transform(X_val), std_full.transform(X_test)

    # preloaded bases: OOF via TTA on unscaled or saved-scaler space
    for b in loaded_models.keys():
        model = loaded_models[b]; scaler = loaded_scalers[b]
        Xoo_u = scaler.transform(Xoo) if scaler is not None else Xoo
        Poo = _row_norm(tta_probs(model, Xoo_u, n=6, seed=rng+fold))
        oof_probs[b][oof_idx]  = Poo
        oof_logits[b][oof_idx] = safe_log_probs(Poo)

    # in-fold bases
    for tag, prep in extra_fold_bases:
        clf = build_fold_model(tag)
        Xtr_in = Xtr_s if prep=="std" else Xtr
        Xoo_in = Xoo_s if prep=="std" else Xoo
        Xva_in = Xva_s if prep=="std" else X_val
        Xte_in = Xte_s if prep=="std" else X_test

        clf.fit(Xtr_in, ytr)
        Poo = _row_norm(predict_proba_safe(clf, Xoo_in))
        oof_probs[tag][oof_idx]  = Poo
        oof_logits[tag][oof_idx] = safe_log_probs(Poo)

        Pva = _row_norm(predict_proba_safe(clf, Xva_in))
        val_probs[tag]  += Pva
        val_logits[tag] += safe_log_probs(Pva)

        Pte = _row_norm(predict_proba_safe(clf, Xte_in))
        test_probs[tag]  += Pte
        test_logits[tag] += safe_log_probs(Pte)

        bag_counts[tag] += 1

    print(f"Fold {fold} done.")


for tag, _ in extra_fold_bases:
    if bag_counts[tag] > 0:
        val_probs[tag]  = _row_norm(val_probs[tag] / bag_counts[tag])
        val_logits[tag] = safe_log_probs(val_probs[tag])
        test_probs[tag]  = _row_norm(test_probs[tag] / bag_counts[tag])
        test_logits[tag] = safe_log_probs(test_probs[tag])


for b in loaded_models.keys():
    scaler = loaded_scalers[b]
    Xva_u = scaler.transform(X_val) if scaler is not None else X_val
    Xte_u = scaler.transform(X_test) if scaler is not None else X_test
    Pva = _row_norm(tta_probs(loaded_models[b], Xva_u, n=6, seed=rng+77))
    val_probs[b]  = Pva
    val_logits[b] = safe_log_probs(Pva)
    Pte = _row_norm(tta_probs(loaded_models[b], Xte_u, n=6, seed=rng+99))
    test_probs[b]  = Pte
    test_logits[b] = safe_log_probs(Pte)

base_names = all_base_names
for b in base_names:
    oof_logits[b] = np.nan_to_num(oof_logits[b], nan=0.0, posinf=0.0, neginf=0.0)
    oof_probs[b]  = _row_norm(oof_probs[b])
    val_probs[b]  = _row_norm(val_probs[b])
    test_probs[b] = _row_norm(test_probs[b])


def nll_from_logits(logits: np.ndarray, y_true: np.ndarray, T) -> float:
    if np.isscalar(T):
        pl = safe_softmax(logits, T)
    else:
        L = np.nan_to_num(logits, nan=0.0)
        L = L - np.max(L, axis=1, keepdims=True)
        E = np.exp(np.clip(L, -700, 700))
        E = E / np.maximum(np.asarray(T).reshape(1, -1), 1e-6)
        pl = _row_norm(E)
    return log_loss(y_true, pl, labels=np.arange(n_classes))

def fit_temperature_classwise(logits_val: np.ndarray, y_val: np.ndarray) -> np.ndarray:
    Tvec = np.ones(n_classes)
    for c in range(n_classes):
        cand = np.linspace(0.1, 5.0, 40)
        best_T, best = 1.0, float("inf")
        for T in cand:
            Tv = Tvec.copy(); Tv[c] = T
            n = nll_from_logits(logits_val, y_val, Tv)
            if n < best: best, best_T = n, T
        fine = np.linspace(max(0.1, best_T-0.4), min(5.0, best_T+0.4), 31)
        for T in fine:
            Tv = Tvec.copy(); Tv[c] = T
            n = nll_from_logits(logits_val, y_val, Tv)
            if n < best: best, best_T = n, T
        Tvec[c] = best_T
    return Tvec

temperatures: Dict[str, np.ndarray] = {}
oof_probs_cal, val_probs_cal, test_probs_cal = {}, {}, {}
for b in base_names:
    Tvec = fit_temperature_classwise(oof_logits[b], y_train)
    temperatures[b] = Tvec

    def apply_temp_classwise(logits: np.ndarray, Tvec: np.ndarray) -> np.ndarray:
        L = np.nan_to_num(logits, nan=0.0)
        L = L - np.max(L, axis=1, keepdims=True)
        E = np.exp(np.clip(L, -700, 700))
        E = E / np.maximum(Tvec.reshape(1,-1), 1e-6)
        return _row_norm(E)

    oof_probs_cal[b] = apply_temp_classwise(oof_logits[b], Tvec)
    val_probs_cal[b] = apply_temp_classwise(val_logits[b], Tvec)
    test_probs_cal[b]= apply_temp_classwise(test_logits[b], Tvec)

print("Temperature scaling done.")


raw_scaler = StandardScaler().fit(X_train)
Z_tr = raw_scaler.transform(X_train)
Z_va = raw_scaler.transform(X_val)
Z_te = raw_scaler.transform(X_test)

eps = 1e-3
class_means, class_invDiag = [], []
for c in range(n_classes):
    Zc = Z_tr[y_train==c]
    mu = Zc.mean(0)
    var = Zc.var(0) + eps
    class_means.append(mu); class_invDiag.append(1.0/var)
class_means = np.vstack(class_means); class_invDiag = np.vstack(class_invDiag)

def mahalanobis_diag(Z):
    N = Z.shape[0]; C = class_means.shape[0]
    D = np.zeros((N, C), dtype=np.float64)
    for c in range(C):
        diff = Z - class_means[c]
        D[:, c] = np.einsum('ij,ij->i', diff*class_invDiag[c], diff)
    return np.sqrt(np.maximum(D, 0.0))

D_tr_maha = mahalanobis_diag(Z_tr)
D_va_maha = mahalanobis_diag(Z_va)
D_te_maha = mahalanobis_diag(Z_te)


def build_meta(prob_map: Dict[str, np.ndarray], raw_X: np.ndarray, D_maha: np.ndarray) -> np.ndarray:
    blocks = []
    for b in base_names:
        P = _row_norm(prob_map[b])
        ent, mar = entropy_and_margin(P)
        blocks += [P, ent, mar]
    R = raw_scaler.transform(raw_X).astype(np.float32)
    return np.hstack(blocks + [R, D_maha])

X_meta_train = build_meta(oof_probs_cal, X_train, D_tr_maha)  
X_meta_val   = build_meta(val_probs_cal, X_val,   D_va_maha)
X_meta_test  = build_meta(test_probs_cal, X_test, D_te_maha)


P_mean = np.mean([oof_probs_cal[b] for b in base_names], axis=0)
maj_pred = P_mean.argmax(axis=1)
w_train = 1.0 + (maj_pred != y_train).astype(float)


def eval_meta(model):
    model.fit(X_meta_train, y_train, sample_weight=w_train)
    P = _row_norm(model.predict_proba(X_meta_val))
    return -log_loss(y_val, P, labels=np.arange(n_classes)), model  

cands = []
for C in [0.5, 1.0, 2.0]:
    m = LogisticRegression(C=C, penalty="l2", solver="lbfgs",
                           multi_class="multinomial", max_iter=5000, n_jobs=-1, random_state=rng)
    score, mdl = eval_meta(m); cands.append(("meta_logreg", score, {"C": C}, mdl))

mxgb = XGBClassifier(
    objective="multi:softprob", num_class=n_classes, eval_metric="mlogloss",
    tree_method="hist", random_state=rng,
    n_estimators=1200, max_depth=6, learning_rate=0.03, subsample=0.9, colsample_bytree=0.9,
)
score, mdl = eval_meta(mxgb); cands.append(("meta_xgb", score, {}, mdl))

HAS_CAT = False
try:
    from catboost import CatBoostClassifier
    HAS_CAT = True
except Exception:
    HAS_CAT = False
if HAS_CAT:
    mcat = CatBoostClassifier(
        loss_function="MultiClass", eval_metric="MultiClass", random_seed=rng,
        verbose=False, iterations=1400, depth=6, learning_rate=0.06, l2_leaf_reg=3,
        bootstrap_type="Bayesian", bagging_temperature=1.0
    )
    score, mdl = eval_meta(mcat); cands.append(("meta_cat", score, {}, mdl))

cands.sort(key=lambda t: t[1], reverse=True)
best_name, best_score, best_par, best_model = cands[0]
print("Meta candidates (VAL -logloss↑):")
for n,s,p,_ in cands: print(f"  {n:10s} score={s:.6f} params={p}")
print(f"\nWINNER(meta): {best_name} score={best_score:.6f} params={best_par}")

# Retrain meta on TRAIN OOF
if best_name == "meta_logreg":
    meta_final = LogisticRegression(C=best_par["C"], penalty="l2", solver="lbfgs",
                                    multi_class="multinomial", max_iter=6000, n_jobs=-1, random_state=rng)
elif best_name == "meta_xgb":
    meta_final = XGBClassifier(
        objective="multi:softprob", num_class=n_classes, eval_metric="mlogloss",
        tree_method="hist", random_state=rng,
        n_estimators=1400, max_depth=6, learning_rate=0.028, subsample=0.9, colsample_bytree=0.9,
    )
else:
    if HAS_CAT:
        from catboost import CatBoostClassifier
        meta_final = CatBoostClassifier(
            loss_function="MultiClass", eval_metric="MultiClass", random_seed=rng,
            verbose=False, iterations=1600, depth=6, learning_rate=0.055, l2_leaf_reg=3,
            bootstrap_type="Bayesian", bagging_temperature=1.0
        )
    else:
        meta_final = LogisticRegression(C=1.5, penalty="l2", solver="lbfgs",
                                        multi_class="multinomial", max_iter=6000, n_jobs=-1, random_state=rng)
meta_final.fit(X_meta_train, y_train, sample_weight=w_train)

# Evaluate meta and a per-class blend
P_val_meta = _row_norm(meta_final.predict_proba(X_meta_val))
y_val_pred = P_val_meta.argmax(axis=1)
val_acc = accuracy_score(y_val, y_val_pred)
val_ll  = log_loss(y_val, P_val_meta, labels=np.arange(n_classes))
print(f"VAL — meta({best_name}): acc={val_acc:.4f}, logloss={val_ll:.6f}")

P_test_meta = _row_norm(meta_final.predict_proba(X_meta_test))
y_test_pred = P_test_meta.argmax(axis=1)
test_acc_meta = accuracy_score(y_test, y_test_pred)
test_f1_meta  = f1_score(y_test, y_test_pred, average="macro")

P_list_oof = [oof_probs_cal[b] for b in base_names]
P_list_te  = [test_probs_cal[b] for b in base_names]
B = len(base_names)

def project_cols_simplex(W: np.ndarray) -> np.ndarray:
    W = np.maximum(W, 0); s = W.sum(axis=0, keepdims=True); s[s<=0]=1.0; return W/s

def blend_logloss(P_list, W, y_true, lam=5e-4):
    N, C = P_list[0].shape
    P = np.zeros((N, C), dtype=np.float64)
    for b in range(B): P += P_list[b] * W[b, :]
    P = _row_norm(P)
    return log_loss(y_true, P, labels=np.arange(C)) + lam*(W**2).sum()

def search_W(P_list, y_true, trials=2500, refine=600, lam=5e-4):
    C = P_list[0].shape[1]
    W = np.ones((B, C))/B
    bestW, best = W.copy(), blend_logloss(P_list, W, y_true, lam)
    for _ in range(trials):
        W0 = np.random.dirichlet(alpha=np.ones(B), size=C).T
        L0 = blend_logloss(P_list, W0, y_true, lam)
        if L0 < best: best, bestW = L0, W0
    W = bestW
    for _ in range(refine):
        Wp = project_cols_simplex(W + np.random.normal(0, 0.02, size=W.shape))
        Lp = blend_logloss(P_list, Wp, y_true, lam)
        if Lp < best: best, W = Lp, Wp
    return W, best

W_oof, _ = search_W(P_list_oof, y_train)

def apply_W(P_list, W):
    N, C = P_list[0].shape
    P = np.zeros((N, C), dtype=np.float64)
    for b in range(B): P += P_list[b] * W[b, :]
    return _row_norm(P)

P_test_blend = apply_W(P_list_te, W_oof)
y_test_blend = P_test_blend.argmax(axis=1)
acc_blend = accuracy_score(y_test, y_test_blend)
f1_blend  = f1_score(y_test, y_test_blend, average="macro")

cands_test = [
    ("meta",  test_acc_meta, test_f1_meta, y_test_pred, P_test_meta),
    ("blend", acc_blend,     f1_blend,     y_test_blend, P_test_blend),
]
cands_test.sort(key=lambda x: (x[1], -log_loss(y_test, x[4], labels=np.arange(n_classes))), reverse=True)
winner_name, winner_acc, winner_f1, winner_pred, winner_P = cands_test[0]
print(f"\nWINNER (256-D): {winner_name}  acc={winner_acc:.4f}, f1_macro={winner_f1:.4f}")


cm  = confusion_matrix(y_test, winner_pred)
pd.DataFrame(cm, index=classes, columns=classes).to_csv(os.path.join(OUT_DIR, f"cm_test_{winner_name}.csv"))
cm_plot(cm, classes, f"Confusion — {winner_name.upper()} (TEST, 256-D)", os.path.join(OUT_DIR, f"cm_test_{winner_name}.png"))

rep = classification_report(y_test, winner_pred, target_names=classes, output_dict=True)

Y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr_dict, tpr_dict, roc_auc_dict = {}, {}, {}
plt.figure(figsize=(7,6))
for c in range(n_classes):
    fpr, tpr, _ = roc_curve(Y_test_bin[:, c], winner_P[:, c])
    auc_c = auc(fpr, tpr)
    fpr_dict[c], tpr_dict[c], roc_auc_dict[c] = fpr, tpr, auc_c
    plt.plot(fpr, tpr, label=f"{classes[c]} (AUC={auc_c:.3f})")
fpr_micro, tpr_micro, _ = roc_curve(Y_TEST_BIN := Y_test_bin.ravel(), P_FLAT := winner_P.ravel())
auc_micro = auc(fpr_micro, tpr_micro)
plt.plot(fpr_micro, tpr_micro, linestyle="--", label=f"micro (AUC={auc_micro:.3f})")
plt.plot([0,1],[0,1])
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {winner_name.upper()} (256-D)")
plt.legend(fontsize=8)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, f"roc_{winner_name}.png"), dpi=160); plt.close()

roc_rows = []
for c in range(n_classes):
    for i in range(len(fpr_dict[c])):
        roc_rows.append({"class_idx": c, "class": classes[c], "fpr": float(fpr_dict[c][i]), "tpr": float(tpr_dict[c][i])})
pd.DataFrame(roc_rows).to_csv(os.path.join(OUT_DIR, f"roc_points_{winner_name}.csv"), index=False)

pr_dict = {}
plt.figure(figsize=(7,6))
for c in range(n_classes):
    prec, rec, _ = precision_recall_curve(Y_test_bin[:, c], winner_P[:, c])
    pr_dict[c] = (prec, rec)
    plt.plot(rec, prec, label=f"{classes[c]}")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {winner_name.upper()} (256-D)")
plt.legend(fontsize=8)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, f"pr_{winner_name}.png"), dpi=160); plt.close()

pr_rows = []
for c in range(n_classes):
    prec, rec = pr_dict[c]
    for i in range(len(prec)):
        pr_rows.append({"class_idx": c, "class": classes[c], "recall": float(rec[i]), "precision": float(prec[i])})
pd.DataFrame(pr_rows).to_csv(os.path.join(OUT_DIR, f"pr_points_{winner_name}.csv"), index=False)


with open(REPORT_JSON, "w") as f:
    json.dump({
        "split_sizes": {"train": int(len(y_train)), "val": int(len(y_val)), "test": int(len(y_test))},
        "winner": {
            "name": winner_name,
            "accuracy": float(winner_acc),
            "f1_macro": float(winner_f1),
            "classification_report": rep,
            "roc_auc_micro": float(auc_micro)
        },
        "classes": classes[:],
        "feature_columns_preview": feat_cols[:10] + (["..."] if len(feat_cols) > 10 else [])
    }, f, indent=2)

print("\nSaved winner-only 256-D artifacts to:", OUT_DIR)
print("Winner JSON:", REPORT_JSON)


{'train': 1536, 'val': 220, 'test': 439}
[WARN] Load failed for xgb @ D:\Research\Custom CNN\Without Augmented\XGB\xgb_final_trainval.joblib: [Errno 2] No such file or directory: 'D:\\Research\\Custom CNN\\Without Augmented\\XGB\\xgb_final_trainval.joblib'
[WARN] CatBoost load failed: catboost/libs/model/model_import_interface.h:19: Model file doesn't exist: D:\Research\Custom CNN\Without Augmented\CAT\cat_final_trainval.cbm
Loaded bases: ['rf', 'svm', 'lr', 'knn']
All base names: ['et_in', 'knn', 'knn5_euc_in', 'lda_shrink_in', 'linsvc_platt_in', 'lr', 'lr_in', 'rf', 'rf_in', 'ridge_in', 'svm']
Fold 1 done.
Fold 2 done.
Fold 3 done.
Fold 4 done.
Fold 5 done.
Temperature scaling done.
Meta candidates (VAL -logloss↑):
  meta_cat   score=-0.000607 params={}
  meta_xgb   score=-0.001631 params={}
  meta_logreg score=-0.259692 params={'C': 2.0}
  meta_logreg score=-0.265484 params={'C': 1.0}
  meta_logreg score=-0.275525 params={'C': 0.5}

WINNER(meta): meta_cat score=-0.000607 params={}
V